# CÓDIGO PARA COMPARAÇÃO AVALIAÇÃO DOS JURADOS x LLMS

## ACURACIA WINNERS

In [22]:
import pandas as pd

def evaluate_predictions(ground_truth_path, predictions_path, excluded_debates, voting_type):
    """
    Avalia a precisão das predições com base nos vencedores previstos e reais.
    
    Args:
        ground_truth_path (str): Caminho para o arquivo CSV de ground truth (hard ou soft voting).
        predictions_path (str): Caminho para o arquivo CSV das predições (LLMs).
        excluded_debates (list): Lista de IDs de debates a serem excluídos.
        voting_type (str): Tipo de votação ('hard' ou 'soft') para indicar o tipo de predictions.
    
    Returns:
        None: Exibe a precisão calculada e detalhes.
    """
    # Carregar datasets
    ground_truth_df = pd.read_csv(ground_truth_path)
    predictions_df = pd.read_csv(predictions_path)

    # Excluir debates específicos
    ground_truth_df = ground_truth_df[~ground_truth_df['debate_id'].isin(excluded_debates)]
    predictions_df = predictions_df[~predictions_df['debate_id'].isin(excluded_debates)]

    # Renomear colunas para garantir consistência
    ground_truth_df = ground_truth_df.rename(columns={
        'name': 'debater_name',
        'position': 'debater_position'
    })

    # Mesclar datasets com base no debate_id
    merged_df = pd.merge(
        ground_truth_df[['debate_id', 'debater_name', 'debater_position']],
        predictions_df[['debate_id', 'debater_name', 'debater_position']],
        on='debate_id',
        suffixes=('_true', '_pred')
    )

    # Identificar vencedores no ground truth (hard ou soft voting)
    ground_truth_winners = merged_df[merged_df['debater_position_true'] == 1]

    # Inicializar lista para cálculos de precisão e detalhes
    correct_predictions = []
    debate_details = []

    for debate_id, group in ground_truth_winners.groupby('debate_id'):
        # Obter os nomes dos vencedores no ground truth
        true_winners = set(group['debater_name_true'])
        
        # Obter os vencedores previstos para este debate_id (LLMs)
        predicted_winners = set(
            merged_df[
                (merged_df['debate_id'] == debate_id) & 
                (merged_df['debater_position_pred'] == 1)
            ]['debater_name_pred']
        )
        
        # Salvar os detalhes para exibição
        debate_details.append({
            'debate_id': debate_id,
            'true_winners': true_winners,
            'predicted_winners': predicted_winners
        })
        
        # Se houver interseção entre os vencedores reais e previstos, é uma predição correta
        if true_winners.intersection(predicted_winners):
            correct_predictions.append(1)
        else:
            correct_predictions.append(0)

    # Calcular a acurácia
    accuracy = sum(correct_predictions) / len(correct_predictions)
    print(f'Acurácia dos vencedores ({voting_type} voting): {accuracy * 100:.2f}%')

    # Exibir os vencedores esperados e previstos para cada debate
    for detail in debate_details:
        print(f"Debate ID: {detail['debate_id']}")
        print(f"Vencedores esperados: {', '.join(detail['true_winners'])}")
        print(f"Vencedores previstos: {', '.join(detail['predicted_winners'])}")
        print('-' * 40)


In [25]:
# Caminhos para os arquivos
ground_truth_hard = 'hard_voting.csv'
ground_truth_soft = 'soft_voting.csv'
predictions_gemini = 'prompt1_gemini.csv'
predictions_gpt = 'prompt1_gpt4o.csv'

# Lista de debates a serem excluídos
excluded_debates = []

# Avaliar para Hard Voting com Gemini
print("Avaliação para Hard Voting com Gemini:")
evaluate_predictions(ground_truth_hard, predictions_gemini, excluded_debates, 'hard')

# Avaliar para Hard Voting com GPT
print("\nAvaliação para Hard Voting com GPT:")
evaluate_predictions(ground_truth_hard, predictions_gpt, excluded_debates, 'hard')

# Avaliar para Soft Voting com Gemini
print("\nAvaliação para Soft Voting com Gemini:")
evaluate_predictions(ground_truth_soft, predictions_gemini, excluded_debates, 'soft')

# Avaliar para Soft Voting com GPT
print("\nAvaliação para Soft Voting com GPT:")
evaluate_predictions(ground_truth_soft, predictions_gpt, excluded_debates, 'soft')


Avaliação para Hard Voting com Gemini:
Acurácia dos vencedores (hard voting): 37.50%
Debate ID: 1
Vencedores esperados: Debater 2
Vencedores previstos: Debater 4
----------------------------------------
Debate ID: 2
Vencedores esperados: Debater 5
Vencedores previstos: Debater 5
----------------------------------------
Debate ID: 3
Vencedores esperados: Debater 3
Vencedores previstos: Debater 4
----------------------------------------
Debate ID: 5
Vencedores esperados: Debater 5
Vencedores previstos: Debater 5
----------------------------------------
Debate ID: 6
Vencedores esperados: Debater 1
Vencedores previstos: Debater 2
----------------------------------------
Debate ID: 7
Vencedores esperados: Debater 2
Vencedores previstos: Debater 4
----------------------------------------
Debate ID: 8
Vencedores esperados: Debater 1
Vencedores previstos: Debater 2
----------------------------------------
Debate ID: 9
Vencedores esperados: Debater 4, Debater 1
Vencedores previstos: Debater 2
-

In [26]:
# Caminhos para os arquivos
ground_truth_hard = 'hard_voting.csv'
ground_truth_soft = 'soft_voting.csv'
predictions_gemini = 'prompt2_gemini.csv'
predictions_gpt = 'prompt2_gpt4o.csv'

# Lista de debates a serem excluídos
excluded_debates = []

# Avaliar para Hard Voting com Gemini
print("Avaliação para Hard Voting com Gemini:")
evaluate_predictions(ground_truth_hard, predictions_gemini, excluded_debates, 'hard')

# Avaliar para Hard Voting com GPT
print("\nAvaliação para Hard Voting com GPT:")
evaluate_predictions(ground_truth_hard, predictions_gpt, excluded_debates, 'hard')

# Avaliar para Soft Voting com Gemini
print("\nAvaliação para Soft Voting com Gemini:")
evaluate_predictions(ground_truth_soft, predictions_gemini, excluded_debates, 'soft')

# Avaliar para Soft Voting com GPT
print("\nAvaliação para Soft Voting com GPT:")
evaluate_predictions(ground_truth_soft, predictions_gpt, excluded_debates, 'soft')

Avaliação para Hard Voting com Gemini:
Acurácia dos vencedores (hard voting): 75.00%
Debate ID: 1
Vencedores esperados: Debater 2
Vencedores previstos: Debater 2
----------------------------------------
Debate ID: 2
Vencedores esperados: Debater 5
Vencedores previstos: Debater 5
----------------------------------------
Debate ID: 3
Vencedores esperados: Debater 3
Vencedores previstos: Debater 4
----------------------------------------
Debate ID: 5
Vencedores esperados: Debater 5
Vencedores previstos: Debater 3
----------------------------------------
Debate ID: 6
Vencedores esperados: Debater 1
Vencedores previstos: Debater 1
----------------------------------------
Debate ID: 7
Vencedores esperados: Debater 2
Vencedores previstos: Debater 4
----------------------------------------
Debate ID: 8
Vencedores esperados: Debater 1
Vencedores previstos: Debater 1
----------------------------------------
Debate ID: 9
Vencedores esperados: Debater 4, Debater 1
Vencedores previstos: Debater 1
-

In [27]:
# Caminhos para os arquivos
ground_truth_hard = 'hard_voting.csv'
ground_truth_soft = 'soft_voting.csv'
predictions_gemini = 'prompt3_gemini.csv'
predictions_gpt = 'prompt3_gpt4o.csv'

# Lista de debates a serem excluídos
excluded_debates = []

# Avaliar para Hard Voting com Gemini
print("Avaliação para Hard Voting com Gemini:")
evaluate_predictions(ground_truth_hard, predictions_gemini, excluded_debates, 'hard')

# Avaliar para Hard Voting com GPT
print("\nAvaliação para Hard Voting com GPT:")
evaluate_predictions(ground_truth_hard, predictions_gpt, excluded_debates, 'hard')

# Avaliar para Soft Voting com Gemini
print("\nAvaliação para Soft Voting com Gemini:")
evaluate_predictions(ground_truth_soft, predictions_gemini, excluded_debates, 'soft')

# Avaliar para Soft Voting com GPT
print("\nAvaliação para Soft Voting com GPT:")
evaluate_predictions(ground_truth_soft, predictions_gpt, excluded_debates, 'soft')

Avaliação para Hard Voting com Gemini:
Acurácia dos vencedores (hard voting): 81.25%
Debate ID: 1
Vencedores esperados: Debater 2
Vencedores previstos: Debater 2, Debater 4
----------------------------------------
Debate ID: 2
Vencedores esperados: Debater 5
Vencedores previstos: Debater 5
----------------------------------------
Debate ID: 3
Vencedores esperados: Debater 3
Vencedores previstos: Debater 3, Debater 5, Debater 4, Debater 1
----------------------------------------
Debate ID: 5
Vencedores esperados: Debater 5
Vencedores previstos: Debater 5, Debater 2, Debater 3, Debater 4, Debater 1
----------------------------------------
Debate ID: 6
Vencedores esperados: Debater 1
Vencedores previstos: Debater 1
----------------------------------------
Debate ID: 7
Vencedores esperados: Debater 2
Vencedores previstos: Debater 3, Debater 2, Debater 4, Debater 1
----------------------------------------
Debate ID: 8
Vencedores esperados: Debater 1
Vencedores previstos: Debater 1
---------

In [28]:
# Caminhos para os arquivos
ground_truth_hard = 'hard_voting.csv'
ground_truth_soft = 'soft_voting.csv'
predictions_gemini = 'prompt4_gemini.csv'
predictions_gpt = 'prompt4_gpt4o.csv'

# Lista de debates a serem excluídos
excluded_debates = []

# Avaliar para Hard Voting com Gemini
print("Avaliação para Hard Voting com Gemini:")
evaluate_predictions(ground_truth_hard, predictions_gemini, excluded_debates, 'hard')

# Avaliar para Hard Voting com GPT
print("\nAvaliação para Hard Voting com GPT:")
evaluate_predictions(ground_truth_hard, predictions_gpt, excluded_debates, 'hard')

# Avaliar para Soft Voting com Gemini
print("\nAvaliação para Soft Voting com Gemini:")
evaluate_predictions(ground_truth_soft, predictions_gemini, excluded_debates, 'soft')

# Avaliar para Soft Voting com GPT
print("\nAvaliação para Soft Voting com GPT:")
evaluate_predictions(ground_truth_soft, predictions_gpt, excluded_debates, 'soft')

Avaliação para Hard Voting com Gemini:
Acurácia dos vencedores (hard voting): 43.75%
Debate ID: 1
Vencedores esperados: Debater 2
Vencedores previstos: Debater 3, Debater 2, Debater 4, Debater 1
----------------------------------------
Debate ID: 2
Vencedores esperados: Debater 5
Vencedores previstos: Debater 5
----------------------------------------
Debate ID: 3
Vencedores esperados: Debater 3
Vencedores previstos: Debater 4
----------------------------------------
Debate ID: 5
Vencedores esperados: Debater 5
Vencedores previstos: Debater 2
----------------------------------------
Debate ID: 6
Vencedores esperados: Debater 1
Vencedores previstos: Debater 2
----------------------------------------
Debate ID: 7
Vencedores esperados: Debater 2
Vencedores previstos: Debater 4, Debater 1
----------------------------------------
Debate ID: 8
Vencedores esperados: Debater 1
Vencedores previstos: Debater 1
----------------------------------------
Debate ID: 9
Vencedores esperados: Debater 4,

## ACURACIA DEBATERS

In [29]:
import pandas as pd

def calculate_ranking_accuracy(ground_truth_df, predictions_df, excluded_debates=[10, 17]):
    """
    Calcula a acurácia do ranking comparando os rankings reais (soft ou hard voting)
    com os rankings previstos pelos modelos LLM (GPT-4, Gemini).
    
    Parameters:
    - ground_truth_df: DataFrame com os rankings reais (soft ou hard voting).
    - predictions_df: DataFrame com as previsões dos modelos (LLMs).
    - excluded_debates: Lista de debates a serem excluídos da análise (default: [10, 17]).
    
    Returns:
    - ranking_accuracy: Acurácia do ranking (em porcentagem).
    """

    # Filtra os debates a serem excluídos
    ground_truth_df = ground_truth_df[~ground_truth_df['debate_id'].isin(excluded_debates)]
    predictions_df = predictions_df[~predictions_df['debate_id'].isin(excluded_debates)]

    # Lista para armazenar as acurácias de ranking de cada debate
    correct_rankings = []
    debate_details = []

    # Variáveis para acumular o número total de acertos e debatedores
    total_correct = 0
    total_debaters = 0

    # Para cada debate, compara o ranking completo dos debatedores
    for debate_id in ground_truth_df['debate_id'].unique():
        # Filtra os debatedores do debate real e do modelo
        true_ranking = ground_truth_df[ground_truth_df['debate_id'] == debate_id][['name', 'position']]
        predicted_ranking = predictions_df[predictions_df['debate_id'] == debate_id][['debater_name', 'debater_position']]

        # Ordena os debatedores conforme suas posições no ranking (ground truth e previsão)
        true_ranking_sorted = true_ranking.sort_values(by='position').reset_index(drop=True)
        predicted_ranking_sorted = predicted_ranking.sort_values(by='debater_position').reset_index(drop=True)

        # Salva os detalhes do debate para exibição
        debate_details.append({
            'debate_id': debate_id,
            'true_ranking': true_ranking_sorted['name'].tolist(),
            'predicted_ranking': predicted_ranking_sorted['debater_name'].tolist()
        })

        # Inicializa o contador de acertos considerando empates
        correct_rankings_count = 0

        # Compara os rankings completos (considerando a ordem exata de posições)
        for i in range(len(true_ranking_sorted)):
            if true_ranking_sorted.iloc[i]['name'] == predicted_ranking_sorted.iloc[i]['debater_name']:
                correct_rankings_count += 1

        # Calcula a fração de debatedores classificados corretamente neste debate
        correct_rankings.append(correct_rankings_count / len(true_ranking_sorted))

        # Acumula os valores totais
        total_correct += correct_rankings_count
        total_debaters += len(true_ranking_sorted)

        # Exibe informações do debate
        print(f"Debate ID: {debate_id}")
        print(f"Ranking esperado: {', '.join(true_ranking_sorted['name'])}")
        print(f"Ranking previsto: {', '.join(predicted_ranking_sorted['debater_name'])}")
        print(f"{correct_rankings_count} acertos de {len(true_ranking_sorted)} debatedores.\n")

    # Calcula a acurácia média de ranking
    ranking_accuracy = sum(correct_rankings) / len(correct_rankings) if correct_rankings else 0

    # Exibe a acurácia do ranking
    print(f'Acurácia de ranking: {ranking_accuracy * 100:.2f}%')

    # Exibe o número total de acertos dividido pelo número total de debatedores
    print(f"Total de acertos: {total_correct} de {total_debaters} debatedores ({(total_correct / total_debaters) * 100:.2f}%)")

    return ranking_accuracy

# Função principal para avaliar hard e soft voting comparados aos LLMs
def evaluate_voting(ground_truth_path, predictions_path, excluded_debates, voting_type):
    print(f"\n--- Avaliando {voting_type.capitalize()} Voting ---\n")

    # Carregar os datasets
    ground_truth_df = pd.read_csv(ground_truth_path)
    predictions_df = pd.read_csv(predictions_path)

    # Ajustar nomes das colunas para uniformizar
    ground_truth_df.rename(columns={'position': 'position', 'name': 'name'}, inplace=True)
    predictions_df.rename(columns={'debater_position': 'debater_position', 'debater_name': 'debater_name'}, inplace=True)

    # Calcular acurácia do ranking
    calculate_ranking_accuracy(ground_truth_df, predictions_df, excluded_debates)



In [31]:
# Caminhos para os arquivos
ground_truth_hard = 'hard_voting.csv'
ground_truth_soft = 'soft_voting.csv'
predictions_gemini = 'prompt1_gemini.csv'
predictions_gpt = 'prompt1_gpt4o.csv'

# Lista de debates a serem excluídos
excluded_debates = []

# Avaliar para Hard Voting com Gemini
print("Avaliação para Hard Voting com Gemini:")
evaluate_voting(ground_truth_hard, predictions_gemini, excluded_debates, 'hard')

# Avaliar para Hard Voting com GPT
print("\nAvaliação para Hard Voting com GPT:")
evaluate_voting(ground_truth_hard, predictions_gpt, excluded_debates, 'hard')

# Avaliar para Soft Voting com Gemini
print("\nAvaliação para Soft Voting com Gemini:")
evaluate_voting(ground_truth_soft, predictions_gemini, excluded_debates, 'soft')

# Avaliar para Soft Voting com GPT
print("\nAvaliação para Soft Voting com GPT:")
evaluate_voting(ground_truth_soft, predictions_gpt, excluded_debates, 'soft')


Avaliação para Hard Voting com Gemini:

--- Avaliando Hard Voting ---

Debate ID: 1
Ranking esperado: Debater 2, Debater 3, Debater 1, Debater 4
Ranking previsto: Debater 4, Debater 2, Debater 3, Debater 1
0 acertos de 4 debatedores.

Debate ID: 2
Ranking esperado: Debater 5, Debater 4, Debater 1, Debater 2, Debater 3
Ranking previsto: Debater 5, Debater 4, Debater 2, Debater 1, Debater 3
3 acertos de 5 debatedores.

Debate ID: 3
Ranking esperado: Debater 3, Debater 4, Debater 1, Debater 2, Debater 5
Ranking previsto: Debater 4, Debater 2, Debater 5, Debater 1, Debater 3
0 acertos de 5 debatedores.

Debate ID: 5
Ranking esperado: Debater 5, Debater 3, Debater 1, Debater 2, Debater 4
Ranking previsto: Debater 5, Debater 3, Debater 2, Debater 4, Debater 1
2 acertos de 5 debatedores.

Debate ID: 6
Ranking esperado: Debater 1, Debater 2, Debater 3, Debater 4
Ranking previsto: Debater 2, Debater 1, Debater 4, Debater 3
0 acertos de 4 debatedores.

Debate ID: 7
Ranking esperado: Debater 2, D

In [32]:
# Caminhos para os arquivos
ground_truth_hard = 'hard_voting.csv'
ground_truth_soft = 'soft_voting.csv'
predictions_gemini = 'prompt2_gemini.csv'
predictions_gpt = 'prompt2_gpt4o.csv'

# Lista de debates a serem excluídos
excluded_debates = []

# Avaliar para Hard Voting com Gemini
print("Avaliação para Hard Voting com Gemini:")
evaluate_voting(ground_truth_hard, predictions_gemini, excluded_debates, 'hard')

# Avaliar para Hard Voting com GPT
print("\nAvaliação para Hard Voting com GPT:")
evaluate_voting(ground_truth_hard, predictions_gpt, excluded_debates, 'hard')

# Avaliar para Soft Voting com Gemini
print("\nAvaliação para Soft Voting com Gemini:")
evaluate_voting(ground_truth_soft, predictions_gemini, excluded_debates, 'soft')

# Avaliar para Soft Voting com GPT
print("\nAvaliação para Soft Voting com GPT:")
evaluate_voting(ground_truth_soft, predictions_gpt, excluded_debates, 'soft')

Avaliação para Hard Voting com Gemini:

--- Avaliando Hard Voting ---

Debate ID: 1
Ranking esperado: Debater 2, Debater 3, Debater 1, Debater 4
Ranking previsto: Debater 2, Debater 1, Debater 3, Debater 4
2 acertos de 4 debatedores.

Debate ID: 2
Ranking esperado: Debater 5, Debater 4, Debater 1, Debater 2, Debater 3
Ranking previsto: Debater 5, Debater 4, Debater 1, Debater 2, Debater 3
5 acertos de 5 debatedores.

Debate ID: 3
Ranking esperado: Debater 3, Debater 4, Debater 1, Debater 2, Debater 5
Ranking previsto: Debater 4, Debater 1, Debater 2, Debater 3, Debater 5
1 acertos de 5 debatedores.

Debate ID: 5
Ranking esperado: Debater 5, Debater 3, Debater 1, Debater 2, Debater 4
Ranking previsto: Debater 3, Debater 2, Debater 5, Debater 1, Debater 4
1 acertos de 5 debatedores.

Debate ID: 6
Ranking esperado: Debater 1, Debater 2, Debater 3, Debater 4
Ranking previsto: Debater 1, Debater 4, Debater 2, Debater 3
1 acertos de 4 debatedores.

Debate ID: 7
Ranking esperado: Debater 2, D

In [33]:
# Caminhos para os arquivos
ground_truth_hard = 'hard_voting.csv'
ground_truth_soft = 'soft_voting.csv'
predictions_gemini = 'prompt3_gemini.csv'
predictions_gpt = 'prompt3_gpt4o.csv'

# Lista de debates a serem excluídos
excluded_debates = []

# Avaliar para Hard Voting com Gemini
print("Avaliação para Hard Voting com Gemini:")
evaluate_voting(ground_truth_hard, predictions_gemini, excluded_debates, 'hard')

# Avaliar para Hard Voting com GPT
print("\nAvaliação para Hard Voting com GPT:")
evaluate_voting(ground_truth_hard, predictions_gpt, excluded_debates, 'hard')

# Avaliar para Soft Voting com Gemini
print("\nAvaliação para Soft Voting com Gemini:")
evaluate_voting(ground_truth_soft, predictions_gemini, excluded_debates, 'soft')

# Avaliar para Soft Voting com GPT
print("\nAvaliação para Soft Voting com GPT:")
evaluate_voting(ground_truth_soft, predictions_gpt, excluded_debates, 'soft')

Avaliação para Hard Voting com Gemini:

--- Avaliando Hard Voting ---

Debate ID: 1
Ranking esperado: Debater 2, Debater 3, Debater 1, Debater 4
Ranking previsto: Debater 2, Debater 4, Debater 3, Debater 1
1 acertos de 4 debatedores.

Debate ID: 2
Ranking esperado: Debater 5, Debater 4, Debater 1, Debater 2, Debater 3
Ranking previsto: Debater 5, Debater 1, Debater 3, Debater 2, Debater 4
2 acertos de 5 debatedores.

Debate ID: 3
Ranking esperado: Debater 3, Debater 4, Debater 1, Debater 2, Debater 5
Ranking previsto: Debater 1, Debater 3, Debater 4, Debater 5, Debater 2
0 acertos de 5 debatedores.

Debate ID: 5
Ranking esperado: Debater 5, Debater 3, Debater 1, Debater 2, Debater 4
Ranking previsto: Debater 1, Debater 2, Debater 3, Debater 4, Debater 5
0 acertos de 5 debatedores.

Debate ID: 6
Ranking esperado: Debater 1, Debater 2, Debater 3, Debater 4
Ranking previsto: Debater 1, Debater 2, Debater 3, Debater 4
4 acertos de 4 debatedores.

Debate ID: 7
Ranking esperado: Debater 2, D

In [34]:
# Caminhos para os arquivos
ground_truth_hard = 'hard_voting.csv'
ground_truth_soft = 'soft_voting.csv'
predictions_gemini = 'prompt4_gemini.csv'
predictions_gpt = 'prompt4_gpt4o.csv'

# Lista de debates a serem excluídos
excluded_debates = []

# Avaliar para Hard Voting com Gemini
print("Avaliação para Hard Voting com Gemini:")
evaluate_voting(ground_truth_hard, predictions_gemini, excluded_debates, 'hard')

# Avaliar para Hard Voting com GPT
print("\nAvaliação para Hard Voting com GPT:")
evaluate_voting(ground_truth_hard, predictions_gpt, excluded_debates, 'hard')

# Avaliar para Soft Voting com Gemini
print("\nAvaliação para Soft Voting com Gemini:")
evaluate_voting(ground_truth_soft, predictions_gemini, excluded_debates, 'soft')

# Avaliar para Soft Voting com GPT
print("\nAvaliação para Soft Voting com GPT:")
evaluate_voting(ground_truth_soft, predictions_gpt, excluded_debates, 'soft')

Avaliação para Hard Voting com Gemini:

--- Avaliando Hard Voting ---

Debate ID: 1
Ranking esperado: Debater 2, Debater 3, Debater 1, Debater 4
Ranking previsto: Debater 1, Debater 2, Debater 3, Debater 4
1 acertos de 4 debatedores.

Debate ID: 2
Ranking esperado: Debater 5, Debater 4, Debater 1, Debater 2, Debater 3
Ranking previsto: Debater 5, Debater 4, Debater 1, Debater 2, Debater 3
5 acertos de 5 debatedores.

Debate ID: 3
Ranking esperado: Debater 3, Debater 4, Debater 1, Debater 2, Debater 5
Ranking previsto: Debater 4, Debater 1, Debater 3, Debater 5, Debater 2
0 acertos de 5 debatedores.

Debate ID: 5
Ranking esperado: Debater 5, Debater 3, Debater 1, Debater 2, Debater 4
Ranking previsto: Debater 2, Debater 3, Debater 1, Debater 4, Debater 5
2 acertos de 5 debatedores.

Debate ID: 6
Ranking esperado: Debater 1, Debater 2, Debater 3, Debater 4
Ranking previsto: Debater 2, Debater 1, Debater 4, Debater 3
0 acertos de 4 debatedores.

Debate ID: 7
Ranking esperado: Debater 2, D

## MRR

In [19]:
import numpy as np

def calculate_mrr(ground_truth_df, predictions_df, exclude_debates=None):
    """
    Calcula o Mean Reciprocal Rank (MRR) para os rankings previstos, utilizando as posições dos debatedores.

    Parameters:
    - ground_truth_df: DataFrame contendo os dados reais (ground truth).
      Deve conter as colunas: 'debate_id', 'debater_name', 'debater_position'.
    - predictions_df: DataFrame contendo as previsões de rankings.
      Deve conter as colunas: 'debate_id', 'debater_name', 'debater_position'.
    - exclude_debates: Lista de IDs de debates a serem excluídos. Default: None.

    Returns:
    - mean_mrr: MRR médio (float).
    """
    # Filtrar debates, se necessário
    if exclude_debates:
        ground_truth_df = ground_truth_df[~ground_truth_df['debate_id'].isin(exclude_debates)]
        predictions_df = predictions_df[~predictions_df['debate_id'].isin(exclude_debates)]

    reciprocal_ranks = []

    # Processar cada debate
    for debate_id in ground_truth_df['debate_id'].unique():
        ground_truth = ground_truth_df[ground_truth_df['debate_id'] == debate_id]
        predictions = predictions_df[predictions_df['debate_id'] == debate_id]

        # Garantir consistência
        ground_truth = ground_truth.sort_values(by='debater_name').reset_index(drop=True)
        predictions = predictions.sort_values(by='debater_name').reset_index(drop=True)

        if len(ground_truth) != len(predictions):
            raise ValueError(f"Debate {debate_id} possui tamanhos inconsistentes entre ground truth e previsões.")

        # Identificar vencedores no ground truth (posição mais alta, 1 é a melhor posição)
        true_winners = set(ground_truth[ground_truth['debater_position'] == min(ground_truth['debater_position'])]['debater_name'])
        rr = 0

        # Calcular Reciprocal Rank
        for rank, debater in enumerate(predictions.sort_values(by='debater_position')['debater_name'], start=1):
            if debater in true_winners:
                rr = 1 / rank
                break

        reciprocal_ranks.append(rr)

        # Debugging: Mostrar informações do debate atual
        print(f"Debate ID: {debate_id}")
        print(f"Vencedores esperados (Position 1): {', '.join(true_winners)}")
        print(f"Reciprocal Rank: {rr:.3f}\n")

    # Calcular MRR médio e retornar
    mean_mrr = np.mean(reciprocal_ranks) if reciprocal_ranks else 0
    return mean_mrr



In [35]:
# Carregando os dados (substitua pelos caminhos corretos dos seus arquivos)
hard_voting_df = pd.read_csv('hard_voting.csv')  # Ground truth para hard voting
soft_voting_df = pd.read_csv('soft_voting.csv')  # Ground truth para soft voting

prompt1_gemini_df = pd.read_csv('prompt1_gemini.csv')
prompt1_gpt_df = pd.read_csv('prompt1_gpt4o.csv')

# Renomeando as colunas de hard_voting e soft_voting para se alinharem com as colunas de predições
hard_voting_df = hard_voting_df.rename(columns={'name': 'debater_name', 'position': 'debater_position', 'score': 'debater_score'})
soft_voting_df = soft_voting_df.rename(columns={'name': 'debater_name', 'position': 'debater_position', 'score': 'debater_score'})

# Debates a excluir
exclude_debates = []

# Cálculo para cada combinação

# 1. prompt1_gemini x hard voting
print("\n### prompt1_gemini x hard voting ###")
mrr_gemini_hard = calculate_mrr(hard_voting_df, prompt1_gemini_df, exclude_debates=exclude_debates)
print(f"MRR: {mrr_gemini_hard:.3f}")

# 2. prompt1_gpt x hard voting
print("\n### prompt1_gpt x hard voting ###")
mrr_gpt_hard = calculate_mrr(hard_voting_df, prompt1_gpt_df, exclude_debates=exclude_debates)
print(f"MRR: {mrr_gpt_hard:.3f}")

# 3. prompt1_gpt x soft voting
print("\n### prompt1_gpt x soft voting ###")
mrr_gpt_soft = calculate_mrr(soft_voting_df, prompt1_gpt_df, exclude_debates=exclude_debates)
print(f"MRR: {mrr_gpt_soft:.3f}")

# 4. prompt1_gemini x soft voting
print("\n### prompt1_gemini x soft voting ###")
mrr_gemini_soft = calculate_mrr(soft_voting_df, prompt1_gemini_df, exclude_debates=exclude_debates)
print(f"MRR: {mrr_gemini_soft:.3f}")



### prompt1_gemini x hard voting ###
Debate ID: 1
Vencedores esperados (Position 1): Debater 2
Reciprocal Rank: 0.500

Debate ID: 2
Vencedores esperados (Position 1): Debater 5
Reciprocal Rank: 1.000

Debate ID: 3
Vencedores esperados (Position 1): Debater 3
Reciprocal Rank: 0.200

Debate ID: 5
Vencedores esperados (Position 1): Debater 5
Reciprocal Rank: 1.000

Debate ID: 6
Vencedores esperados (Position 1): Debater 1
Reciprocal Rank: 0.500

Debate ID: 7
Vencedores esperados (Position 1): Debater 2
Reciprocal Rank: 0.333

Debate ID: 8
Vencedores esperados (Position 1): Debater 1
Reciprocal Rank: 0.500

Debate ID: 9
Vencedores esperados (Position 1): Debater 4, Debater 1
Reciprocal Rank: 0.500

Debate ID: 10
Vencedores esperados (Position 1): Debater 4
Reciprocal Rank: 0.500

Debate ID: 11
Vencedores esperados (Position 1): Debater 5
Reciprocal Rank: 1.000

Debate ID: 12
Vencedores esperados (Position 1): Debater 3
Reciprocal Rank: 1.000

Debate ID: 13
Vencedores esperados (Position 1

In [36]:
# Carregando os dados (substitua pelos caminhos corretos dos seus arquivos)
hard_voting_df = pd.read_csv('hard_voting.csv')  # Ground truth para hard voting
soft_voting_df = pd.read_csv('soft_voting.csv')  # Ground truth para soft voting

prompt1_gemini_df = pd.read_csv('prompt2_gemini.csv')
prompt1_gpt_df = pd.read_csv('prompt2_gpt4o.csv')

# Renomeando as colunas de hard_voting e soft_voting para se alinharem com as colunas de predições
hard_voting_df = hard_voting_df.rename(columns={'name': 'debater_name', 'position': 'debater_position', 'score': 'debater_score'})
soft_voting_df = soft_voting_df.rename(columns={'name': 'debater_name', 'position': 'debater_position', 'score': 'debater_score'})

# Debates a excluir
exclude_debates = []

# Cálculo para cada combinação

# 1. prompt1_gemini x hard voting
print("\n### prompt1_gemini x hard voting ###")
mrr_gemini_hard = calculate_mrr(hard_voting_df, prompt1_gemini_df, exclude_debates=exclude_debates)
print(f"MRR: {mrr_gemini_hard:.3f}")

# 2. prompt1_gpt x hard voting
print("\n### prompt1_gpt x hard voting ###")
mrr_gpt_hard = calculate_mrr(hard_voting_df, prompt1_gpt_df, exclude_debates=exclude_debates)
print(f"MRR: {mrr_gpt_hard:.3f}")

# 3. prompt1_gemini x soft voting
print("\n### prompt1_gemini x soft voting ###")
mrr_gemini_soft = calculate_mrr(soft_voting_df, prompt1_gemini_df, exclude_debates=exclude_debates)
print(f"MRR: {mrr_gemini_soft:.3f}")

# 4. prompt1_gpt x soft voting
print("\n### prompt1_gpt x soft voting ###")
mrr_gpt_soft = calculate_mrr(soft_voting_df, prompt1_gpt_df, exclude_debates=exclude_debates)
print(f"MRR: {mrr_gpt_soft:.3f}")



### prompt1_gemini x hard voting ###
Debate ID: 1
Vencedores esperados (Position 1): Debater 2
Reciprocal Rank: 1.000

Debate ID: 2
Vencedores esperados (Position 1): Debater 5
Reciprocal Rank: 1.000

Debate ID: 3
Vencedores esperados (Position 1): Debater 3
Reciprocal Rank: 0.250

Debate ID: 5
Vencedores esperados (Position 1): Debater 5
Reciprocal Rank: 0.333

Debate ID: 6
Vencedores esperados (Position 1): Debater 1
Reciprocal Rank: 1.000

Debate ID: 7
Vencedores esperados (Position 1): Debater 2
Reciprocal Rank: 0.500

Debate ID: 8
Vencedores esperados (Position 1): Debater 1
Reciprocal Rank: 1.000

Debate ID: 9
Vencedores esperados (Position 1): Debater 4, Debater 1
Reciprocal Rank: 1.000

Debate ID: 10
Vencedores esperados (Position 1): Debater 4
Reciprocal Rank: 1.000

Debate ID: 11
Vencedores esperados (Position 1): Debater 5
Reciprocal Rank: 1.000

Debate ID: 12
Vencedores esperados (Position 1): Debater 3
Reciprocal Rank: 1.000

Debate ID: 13
Vencedores esperados (Position 1

In [37]:
# Carregando os dados (substitua pelos caminhos corretos dos seus arquivos)
hard_voting_df = pd.read_csv('hard_voting.csv')  # Ground truth para hard voting
soft_voting_df = pd.read_csv('soft_voting.csv')  # Ground truth para soft voting

prompt1_gemini_df = pd.read_csv('prompt3_gemini.csv')
prompt1_gpt_df = pd.read_csv('prompt3_gpt4o.csv')

# Renomeando as colunas de hard_voting e soft_voting para se alinharem com as colunas de predições
hard_voting_df = hard_voting_df.rename(columns={'name': 'debater_name', 'position': 'debater_position', 'score': 'debater_score'})
soft_voting_df = soft_voting_df.rename(columns={'name': 'debater_name', 'position': 'debater_position', 'score': 'debater_score'})

# Debates a excluir
exclude_debates = []

# Cálculo para cada combinação

# 1. prompt1_gemini x hard voting
print("\n### prompt1_gemini x hard voting ###")
mrr_gemini_hard = calculate_mrr(hard_voting_df, prompt1_gemini_df, exclude_debates=exclude_debates)
print(f"MRR: {mrr_gemini_hard:.3f}")

# 2. prompt1_gpt x hard voting
print("\n### prompt1_gpt x hard voting ###")
mrr_gpt_hard = calculate_mrr(hard_voting_df, prompt1_gpt_df, exclude_debates=exclude_debates)
print(f"MRR: {mrr_gpt_hard:.3f}")

# 3. prompt1_gemini x soft voting
print("\n### prompt1_gemini x soft voting ###")
mrr_gemini_soft = calculate_mrr(soft_voting_df, prompt1_gemini_df, exclude_debates=exclude_debates)
print(f"MRR: {mrr_gemini_soft:.3f}")

# 4. prompt1_gpt x soft voting
print("\n### prompt1_gpt x soft voting ###")
mrr_gpt_soft = calculate_mrr(soft_voting_df, prompt1_gpt_df, exclude_debates=exclude_debates)
print(f"MRR: {mrr_gpt_soft:.3f}")



### prompt1_gemini x hard voting ###
Debate ID: 1
Vencedores esperados (Position 1): Debater 2
Reciprocal Rank: 1.000

Debate ID: 2
Vencedores esperados (Position 1): Debater 5
Reciprocal Rank: 1.000

Debate ID: 3
Vencedores esperados (Position 1): Debater 3
Reciprocal Rank: 0.500

Debate ID: 5
Vencedores esperados (Position 1): Debater 5
Reciprocal Rank: 0.200

Debate ID: 6
Vencedores esperados (Position 1): Debater 1
Reciprocal Rank: 1.000

Debate ID: 7
Vencedores esperados (Position 1): Debater 2
Reciprocal Rank: 0.500

Debate ID: 8
Vencedores esperados (Position 1): Debater 1
Reciprocal Rank: 1.000

Debate ID: 9
Vencedores esperados (Position 1): Debater 4, Debater 1
Reciprocal Rank: 1.000

Debate ID: 10
Vencedores esperados (Position 1): Debater 4
Reciprocal Rank: 0.250

Debate ID: 11
Vencedores esperados (Position 1): Debater 5
Reciprocal Rank: 0.500

Debate ID: 12
Vencedores esperados (Position 1): Debater 3
Reciprocal Rank: 1.000

Debate ID: 13
Vencedores esperados (Position 1

In [38]:
# Carregando os dados (substitua pelos caminhos corretos dos seus arquivos)
hard_voting_df = pd.read_csv('hard_voting.csv')  # Ground truth para hard voting
soft_voting_df = pd.read_csv('soft_voting.csv')  # Ground truth para soft voting

prompt1_gemini_df = pd.read_csv('prompt4_gemini.csv')
prompt1_gpt_df = pd.read_csv('prompt4_gpt4o.csv')

# Renomeando as colunas de hard_voting e soft_voting para se alinharem com as colunas de predições
hard_voting_df = hard_voting_df.rename(columns={'name': 'debater_name', 'position': 'debater_position', 'score': 'debater_score'})
soft_voting_df = soft_voting_df.rename(columns={'name': 'debater_name', 'position': 'debater_position', 'score': 'debater_score'})

# Debates a excluir
exclude_debates = []

# Cálculo para cada combinação

# 1. prompt1_gemini x hard voting
print("\n### prompt1_gemini x hard voting ###")
mrr_gemini_hard = calculate_mrr(hard_voting_df, prompt1_gemini_df, exclude_debates=exclude_debates)
print(f"MRR: {mrr_gemini_hard:.3f}")

# 2. prompt1_gpt x hard voting
print("\n### prompt1_gpt x hard voting ###")
mrr_gpt_hard = calculate_mrr(hard_voting_df, prompt1_gpt_df, exclude_debates=exclude_debates)
print(f"MRR: {mrr_gpt_hard:.3f}")

# 3. prompt1_gemini x soft voting
print("\n### prompt1_gemini x soft voting ###")
mrr_gemini_soft = calculate_mrr(soft_voting_df, prompt1_gemini_df, exclude_debates=exclude_debates)
print(f"MRR: {mrr_gemini_soft:.3f}")

# 4. prompt1_gpt x soft voting
print("\n### prompt1_gpt x soft voting ###")
mrr_gpt_soft = calculate_mrr(soft_voting_df, prompt1_gpt_df, exclude_debates=exclude_debates)
print(f"MRR: {mrr_gpt_soft:.3f}")


### prompt1_gemini x hard voting ###
Debate ID: 1
Vencedores esperados (Position 1): Debater 2
Reciprocal Rank: 0.500

Debate ID: 2
Vencedores esperados (Position 1): Debater 5
Reciprocal Rank: 1.000

Debate ID: 3
Vencedores esperados (Position 1): Debater 3
Reciprocal Rank: 0.333

Debate ID: 5
Vencedores esperados (Position 1): Debater 5
Reciprocal Rank: 0.200

Debate ID: 6
Vencedores esperados (Position 1): Debater 1
Reciprocal Rank: 0.500

Debate ID: 7
Vencedores esperados (Position 1): Debater 2
Reciprocal Rank: 0.333

Debate ID: 8
Vencedores esperados (Position 1): Debater 1
Reciprocal Rank: 1.000

Debate ID: 9
Vencedores esperados (Position 1): Debater 4, Debater 1
Reciprocal Rank: 1.000

Debate ID: 10
Vencedores esperados (Position 1): Debater 4
Reciprocal Rank: 1.000

Debate ID: 11
Vencedores esperados (Position 1): Debater 5
Reciprocal Rank: 0.500

Debate ID: 12
Vencedores esperados (Position 1): Debater 3
Reciprocal Rank: 1.000

Debate ID: 13
Vencedores esperados (Position 1

## NDCG

In [1]:
from sklearn.metrics import ndcg_score
import numpy as np

def calculate_ndcg(ground_truth_df, predictions_df, k=None, exclude_debates=None):
    """
    Calcula o Mean nDCG para os rankings previstos, utilizando os scores do GT e posições previstas pelos modelos.

    Parameters:
    - ground_truth_df: DataFrame contendo os dados reais (ground truth).
      Deve conter as colunas: 'debate_id', 'position', 'name', 'score'.
    - predictions_df: DataFrame contendo as previsões de rankings.
      Deve conter as colunas: 'debate_id', 'debater_name', 'debater_position', 'debater_score'.
    - k: Inteiro opcional para limitar o cálculo ao top-k. Default: None.
    - exclude_debates: Lista de IDs de debates a serem excluídos. Default: None.

    Returns:
    - mean_ndcg: nDCG médio (float).
    """
    # Filtrar debates, se necessário
    if exclude_debates:
        ground_truth_df = ground_truth_df[~ground_truth_df['debate_id'].isin(exclude_debates)]
        predictions_df = predictions_df[~predictions_df['debate_id'].isin(exclude_debates)]

    ndcg_scores = []

    for debate_id in ground_truth_df['debate_id'].unique():
        ground_truth = ground_truth_df[ground_truth_df['debate_id'] == debate_id]
        predictions = predictions_df[predictions_df['debate_id'] == debate_id]

        # Garantir consistência
        ground_truth = ground_truth.sort_values(by='name').reset_index(drop=True)
        predictions = predictions.sort_values(by='debater_name').reset_index(drop=True)

        if len(ground_truth) != len(predictions):
            raise ValueError(f"Debate {debate_id} possui tamanhos inconsistentes entre ground truth e previsões.")

        # Relevâncias: usar os scores do GT e ordenar conforme as posições previstas
        true_relevance = ground_truth['score'].values
        pred_order = predictions.sort_values(by='debater_position')['debater_name'].values

        # Reordenar relevâncias do GT com base nas posições previstas
        reordered_true_relevance = [true_relevance[list(ground_truth['name']).index(name)] for name in pred_order]

        # Debugging: Print values to ensure they are correct
        print(f"Debate ID: {debate_id}")
        print(f"True Relevance (original): {true_relevance}")
        print(f"Predicted Order: {pred_order}")
        print(f"Reordered True Relevance: {reordered_true_relevance}")

        ndcg = ndcg_score([true_relevance], [reordered_true_relevance], k=k)
        ndcg_scores.append(ndcg)

        print(f"nDCG for Debate {debate_id}: {ndcg:.3f}\n")

    mean_ndcg = np.mean(ndcg_scores) if ndcg_scores else 0

    # Debugging: Print final results
    print(f"All nDCG Scores: {ndcg_scores}")
    print(f"Mean nDCG: {mean_ndcg:.3f}")
    return mean_ndcg




In [3]:
import pandas as pd

# Carregando os dados (substitua pelos caminhos corretos dos seus arquivos)
hard_voting_df = pd.read_csv('hard_voting.csv')  # Ground truth para hard voting
soft_voting_df = pd.read_csv('soft_voting.csv')  # Ground truth para soft voting

prompt1_gemini_df = pd.read_csv('prompt1_gemini.csv')
prompt1_gpt_df = pd.read_csv('prompt1_gpt4o.csv')

# Renomeando as colunas de hard_voting e soft_voting para se alinharem com as colunas de predições
#hard_voting_df = hard_voting_df.rename(columns={'name': 'debater_name', 'position': 'debater_position', 'score': 'debater_score'})
#soft_voting_df = soft_voting_df.rename(columns={'name': 'debater_name', 'position': 'debater_position', 'score': 'debater_score'})

# Debates a excluir
exclude_debates = []

# Cálculo para cada combinação

# 1. prompt1_gemini x hard voting
print("\n### prompt1_gemini x hard voting ###")
ndcg_gemini_hard = calculate_ndcg(hard_voting_df, prompt1_gemini_df, exclude_debates=exclude_debates, k=5)
print({ndcg_gemini_hard})
print(f"nDCG: {ndcg_gemini_hard:.3f}")

# 2. prompt1_gpt x hard voting
print("\n### prompt1_gpt x hard voting ###")
ndcg_gpt_hard = calculate_ndcg(hard_voting_df, prompt1_gpt_df, exclude_debates=exclude_debates, k=5)
print(f"nDCG: {ndcg_gpt_hard:.3f}")

# 3. prompt1_gpt x soft voting
print("\n### prompt1_gpt x soft voting ###")
ndcg_gpt_soft = calculate_ndcg(soft_voting_df, prompt1_gpt_df, exclude_debates=exclude_debates, k=5)
print(f"nDCG: {ndcg_gpt_soft:.3f}")

# 4. prompt1_gemini x soft voting
print("\n### prompt1_gemini x soft voting ###")
ndcg_gemini_soft = calculate_ndcg(soft_voting_df, prompt1_gemini_df, exclude_debates=exclude_debates, k=5)
print(f"nDCG: {ndcg_gemini_soft:.3f}")




### prompt1_gemini x hard voting ###
Debate ID: 1
True Relevance (original): [0 3 2 0]
Predicted Order: ['Debater 4' 'Debater 2' 'Debater 3' 'Debater 1']
Reordered True Relevance: [np.int64(0), np.int64(3), np.int64(2), np.int64(0)]
nDCG for Debate 1: 1.000

Debate ID: 2
True Relevance (original): [0 0 0 2 5]
Predicted Order: ['Debater 5' 'Debater 4' 'Debater 2' 'Debater 1' 'Debater 3']
Reordered True Relevance: [np.int64(5), np.int64(2), np.int64(0), np.int64(0), np.int64(0)]
nDCG for Debate 2: 0.491

Debate ID: 3
True Relevance (original): [1 0 3 2 0]
Predicted Order: ['Debater 4' 'Debater 2' 'Debater 5' 'Debater 1' 'Debater 3']
Reordered True Relevance: [np.int64(2), np.int64(0), np.int64(0), np.int64(1), np.int64(3)]
nDCG for Debate 3: 0.600

Debate ID: 5
True Relevance (original): [0 0 2 0 4]
Predicted Order: ['Debater 5' 'Debater 3' 'Debater 2' 'Debater 4' 'Debater 1']
Reordered True Relevance: [np.int64(4), np.int64(2), np.int64(0), np.int64(0), np.int64(0)]
nDCG for Debate 5: 

In [4]:
import pandas as pd

# Carregando os dados (substitua pelos caminhos corretos dos seus arquivos)
hard_voting_df = pd.read_csv('hard_voting.csv')  # Ground truth para hard voting
soft_voting_df = pd.read_csv('soft_voting.csv')  # Ground truth para soft voting

prompt1_gemini_df = pd.read_csv('prompt2_gemini.csv')
prompt1_gpt_df = pd.read_csv('prompt2_gpt4o.csv')

# Renomeando as colunas de hard_voting e soft_voting para se alinharem com as colunas de predições
#hard_voting_df = hard_voting_df.rename(columns={'name': 'debater_name', 'position': 'debater_position', 'score': 'debater_score'})
#soft_voting_df = soft_voting_df.rename(columns={'name': 'debater_name', 'position': 'debater_position', 'score': 'debater_score'})

# Debates a excluir
exclude_debates = []

# Cálculo para cada combinação

# 1. prompt1_gemini x hard voting
print("\n### prompt1_gemini x hard voting ###")
ndcg_gemini_hard = calculate_ndcg(hard_voting_df, prompt1_gemini_df, exclude_debates=exclude_debates, k=5)
print({ndcg_gemini_hard})
print(f"nDCG: {ndcg_gemini_hard:.3f}")

# 2. prompt1_gpt x hard voting
print("\n### prompt1_gpt x hard voting ###")
ndcg_gpt_hard = calculate_ndcg(hard_voting_df, prompt1_gpt_df, exclude_debates=exclude_debates, k=5)
print(f"nDCG: {ndcg_gpt_hard:.3f}")

# 3. prompt1_gpt x soft voting
print("\n### prompt1_gpt x soft voting ###")
ndcg_gpt_soft = calculate_ndcg(soft_voting_df, prompt1_gpt_df, exclude_debates=exclude_debates, k=5)
print(f"nDCG: {ndcg_gpt_soft:.3f}")

# 4. prompt1_gemini x soft voting
print("\n### prompt1_gemini x soft voting ###")
ndcg_gemini_soft = calculate_ndcg(soft_voting_df, prompt1_gemini_df, exclude_debates=exclude_debates, k=5)
print(f"nDCG: {ndcg_gemini_soft:.3f}")


### prompt1_gemini x hard voting ###
Debate ID: 1
True Relevance (original): [0 3 2 0]
Predicted Order: ['Debater 2' 'Debater 1' 'Debater 3' 'Debater 4']
Reordered True Relevance: [np.int64(3), np.int64(0), np.int64(2), np.int64(0)]
nDCG for Debate 1: 0.624

Debate ID: 2
True Relevance (original): [0 0 0 2 5]
Predicted Order: ['Debater 5' 'Debater 4' 'Debater 1' 'Debater 2' 'Debater 3']
Reordered True Relevance: [np.int64(5), np.int64(2), np.int64(0), np.int64(0), np.int64(0)]
nDCG for Debate 2: 0.491

Debate ID: 3
True Relevance (original): [1 0 3 2 0]
Predicted Order: ['Debater 4' 'Debater 1' 'Debater 2' 'Debater 3' 'Debater 5']
Reordered True Relevance: [np.int64(2), np.int64(1), np.int64(0), np.int64(3), np.int64(0)]
nDCG for Debate 3: 0.810

Debate ID: 5
True Relevance (original): [0 0 2 0 4]
Predicted Order: ['Debater 3' 'Debater 2' 'Debater 5' 'Debater 1' 'Debater 4']
Reordered True Relevance: [np.int64(2), np.int64(0), np.int64(4), np.int64(0), np.int64(0)]
nDCG for Debate 5: 

In [5]:
import pandas as pd

# Carregando os dados (substitua pelos caminhos corretos dos seus arquivos)
hard_voting_df = pd.read_csv('hard_voting.csv')  # Ground truth para hard voting
soft_voting_df = pd.read_csv('soft_voting.csv')  # Ground truth para soft voting

prompt1_gemini_df = pd.read_csv('prompt3_gemini.csv')
prompt1_gpt_df = pd.read_csv('prompt3_gpt4o.csv')

# Renomeando as colunas de hard_voting e soft_voting para se alinharem com as colunas de predições
#hard_voting_df = hard_voting_df.rename(columns={'name': 'debater_name', 'position': 'debater_position', 'score': 'debater_score'})
#soft_voting_df = soft_voting_df.rename(columns={'name': 'debater_name', 'position': 'debater_position', 'score': 'debater_score'})

# Debates a excluir
exclude_debates = []

# Cálculo para cada combinação

# 1. prompt1_gemini x hard voting
print("\n### prompt1_gemini x hard voting ###")
ndcg_gemini_hard = calculate_ndcg(hard_voting_df, prompt1_gemini_df, exclude_debates=exclude_debates, k=5)
print({ndcg_gemini_hard})
print(f"nDCG: {ndcg_gemini_hard:.3f}")

# 2. prompt1_gpt x hard voting
print("\n### prompt1_gpt x hard voting ###")
ndcg_gpt_hard = calculate_ndcg(hard_voting_df, prompt1_gpt_df, exclude_debates=exclude_debates, k=5)
print(f"nDCG: {ndcg_gpt_hard:.3f}")

# 3. prompt1_gpt x soft voting
print("\n### prompt1_gpt x soft voting ###")
ndcg_gpt_soft = calculate_ndcg(soft_voting_df, prompt1_gpt_df, exclude_debates=exclude_debates, k=5)
print(f"nDCG: {ndcg_gpt_soft:.3f}")

# 4. prompt1_gemini x soft voting
print("\n### prompt1_gemini x soft voting ###")
ndcg_gemini_soft = calculate_ndcg(soft_voting_df, prompt1_gemini_df, exclude_debates=exclude_debates, k=5)
print(f"nDCG: {ndcg_gemini_soft:.3f}")


### prompt1_gemini x hard voting ###
Debate ID: 1
True Relevance (original): [0 3 2 0]
Predicted Order: ['Debater 2' 'Debater 4' 'Debater 3' 'Debater 1']
Reordered True Relevance: [np.int64(3), np.int64(0), np.int64(2), np.int64(0)]
nDCG for Debate 1: 0.624

Debate ID: 2
True Relevance (original): [0 0 0 2 5]
Predicted Order: ['Debater 5' 'Debater 1' 'Debater 3' 'Debater 2' 'Debater 4']
Reordered True Relevance: [np.int64(5), np.int64(0), np.int64(0), np.int64(0), np.int64(2)]
nDCG for Debate 2: 0.644

Debate ID: 3
True Relevance (original): [1 0 3 2 0]
Predicted Order: ['Debater 1' 'Debater 3' 'Debater 4' 'Debater 5' 'Debater 2']
Reordered True Relevance: [np.int64(1), np.int64(3), np.int64(2), np.int64(0), np.int64(0)]
nDCG for Debate 3: 0.674

Debate ID: 5
True Relevance (original): [0 0 2 0 4]
Predicted Order: ['Debater 1' 'Debater 2' 'Debater 3' 'Debater 4' 'Debater 5']
Reordered True Relevance: [np.int64(0), np.int64(0), np.int64(2), np.int64(0), np.int64(4)]
nDCG for Debate 5: 

In [6]:
import pandas as pd

# Carregando os dados (substitua pelos caminhos corretos dos seus arquivos)
hard_voting_df = pd.read_csv('hard_voting.csv')  # Ground truth para hard voting
soft_voting_df = pd.read_csv('soft_voting.csv')  # Ground truth para soft voting

prompt1_gemini_df = pd.read_csv('prompt4_gemini.csv')
prompt1_gpt_df = pd.read_csv('prompt4_gpt4o.csv')

# Renomeando as colunas de hard_voting e soft_voting para se alinharem com as colunas de predições
#hard_voting_df = hard_voting_df.rename(columns={'name': 'debater_name', 'position': 'debater_position', 'score': 'debater_score'})
#soft_voting_df = soft_voting_df.rename(columns={'name': 'debater_name', 'position': 'debater_position', 'score': 'debater_score'})

# Debates a excluir
exclude_debates = []

# Cálculo para cada combinação

# 1. prompt1_gemini x hard voting
print("\n### prompt1_gemini x hard voting ###")
ndcg_gemini_hard = calculate_ndcg(hard_voting_df, prompt1_gemini_df, exclude_debates=exclude_debates, k=5)
print({ndcg_gemini_hard})
print(f"nDCG: {ndcg_gemini_hard:.3f}")

# 2. prompt1_gpt x hard voting
print("\n### prompt1_gpt x hard voting ###")
ndcg_gpt_hard = calculate_ndcg(hard_voting_df, prompt1_gpt_df, exclude_debates=exclude_debates, k=5)
print(f"nDCG: {ndcg_gpt_hard:.3f}")

# 3. prompt1_gpt x soft voting
print("\n### prompt1_gpt x soft voting ###")
ndcg_gpt_soft = calculate_ndcg(soft_voting_df, prompt1_gpt_df, exclude_debates=exclude_debates, k=5)
print(f"nDCG: {ndcg_gpt_soft:.3f}")

# 4. prompt1_gemini x soft voting
print("\n### prompt1_gemini x soft voting ###")
ndcg_gemini_soft = calculate_ndcg(soft_voting_df, prompt1_gemini_df, exclude_debates=exclude_debates, k=5)
print(f"nDCG: {ndcg_gemini_soft:.3f}")


### prompt1_gemini x hard voting ###
Debate ID: 1
True Relevance (original): [0 3 2 0]
Predicted Order: ['Debater 1' 'Debater 2' 'Debater 3' 'Debater 4']
Reordered True Relevance: [np.int64(0), np.int64(3), np.int64(2), np.int64(0)]
nDCG for Debate 1: 1.000

Debate ID: 2
True Relevance (original): [0 0 0 2 5]
Predicted Order: ['Debater 5' 'Debater 4' 'Debater 1' 'Debater 2' 'Debater 3']
Reordered True Relevance: [np.int64(5), np.int64(2), np.int64(0), np.int64(0), np.int64(0)]
nDCG for Debate 2: 0.491

Debate ID: 3
True Relevance (original): [1 0 3 2 0]
Predicted Order: ['Debater 4' 'Debater 1' 'Debater 3' 'Debater 5' 'Debater 2']
Reordered True Relevance: [np.int64(2), np.int64(1), np.int64(3), np.int64(0), np.int64(0)]
nDCG for Debate 3: 0.934

Debate ID: 5
True Relevance (original): [0 0 2 0 4]
Predicted Order: ['Debater 2' 'Debater 3' 'Debater 1' 'Debater 4' 'Debater 5']
Reordered True Relevance: [np.int64(0), np.int64(2), np.int64(0), np.int64(0), np.int64(4)]
nDCG for Debate 5: 